In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.relativedelta import relativedelta
import datetime
import statsmodels.api as sm
from scipy.optimize import brute
from itertools import product



# format notebook output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML, Javascript
display(HTML("<style>.container { width:90% !important; }</style>"))

# style pandas display
pd.set_option('display.max_columns', None)

# matplotlib magic
%matplotlib inline

/Users/lee/kaggleVenv/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
air_reserve = pd.read_csv('../../data_files/air_reserve.csv')
air_store_info = pd.read_csv('../../data_files/air_store_info.csv', encoding='utf-8')
air_visit_data = pd.read_csv('../../data_files/air_visit_data.csv')
date_info = pd.read_csv('../../data_files/date_info.csv')
hpg_reserve = pd.read_csv('../../data_files/hpg_reserve.csv')
hpg_store_info = pd.read_csv('../../data_files/hpg_store_info.csv')
sample_submission = pd.read_csv('../../data_files/sample_submission.csv')
store_id_relation = pd.read_csv('../../data_files/store_id_relation.csv')
competition_id_list = []
with open('./competition_id_list.txt') as fp: 
  for line in fp.readlines():
    competition_id_list.append(line)
competition_id_list = [x.strip() for x in competition_id_list]

In [3]:
def fill_missing_dates(df, date_column, id_column, value_column, fill_value, begin_date=None, end_date=None):
    """
    helper function to fill in dates that are missing from the data set
    in order to have a contiguous segment of dates for each location id
    """
    
    if begin_date:
        min_date = begin_date
    else:
        min_date = df[date_column].min()
        
    if end_date:
        max_date = end_date
    else:
        max_date = df[date_column].max()
        
    df.index = pd.DatetimeIndex(df['visit_date'])
    date_df = pd.DataFrame(index=pd.date_range(df.index[0], df.index[-1]))
    date_df['empty'] = np.nan
    join = date_df.join(df,how='outer')[[id_column,value_column]]
    join[id_column]=[join[id_column].values[0]]*len(join)
    
    return join

def grab_air_visit_data_by_id(store_id):
  store_visits = air_visit_data[air_visit_data['air_store_id']==store_id]
  return fill_missing_dates(store_visits,'visit_date','air_store_id','visitors','NA')

In [7]:
data_col = 'visitors'

def build_model(iter_param, series, params_list, static_param=None):
  if static_param==None:
    ords = iter_param
    sords = (1,0,0,7)
  else:
    ords = static_param
    sords = iter_param
  try:
    mod = sm.tsa.statespace.SARIMAX(
        series, trend='n', order=ords, seasonal_order=sords
      )
    res = mod.fit(disp=0)
    params_list.append(tuple((ords, sords, res.aic)))
  except:
    pass

'''m = []
res = build_model((1,0,0), test[data_col], m)
print('AIC: %s' % res.aic)
begin = len(test)-1
end = (datetime.datetime(2017,5,30).date()-test.index[-1].date()).days+len(test)
#res.predict(start=begin, end=end, dynamic=True)'''

"m = []\nres = build_model((1,0,0), test[data_col], m)\nprint('AIC: %s' % res.aic)\nbegin = len(test)-1\nend = (datetime.datetime(2017,5,30).date()-test.index[-1].date()).days+len(test)\n#res.predict(start=begin, end=end, dynamic=True)"

In [5]:
def parameter_search2(series, num_models, grid_diameter=3):
  models = []
  param_vals = range(grid_diameter)
  grid = list(product(param_vals,param_vals,param_vals))
  for ord_param in grid:
    for sord_param in [(p[0],p[1],p[2],7) for p in grid]:
      build_model(sord_param, series, models, ord_param)
  return sorted(models, key=lambda x: x[2])[:num_models]

In [6]:
def parameter_search(series, num_models, grid_diameter=3):
  ord_models = []
  sord_models = []
  grid = (slice(0, grid_diameter, 1), slice(0, grid_diameter, 1), slice(0, grid_diameter, 1))
  brute(build_model, grid, args=(series, ord_models), finish=None)
  for model in sorted(ord_models, key=lambda x: x[2])[:num_models]:
    grid = (slice(0, grid_diameter, 1), slice(0, grid_diameter, 1), slice(0, grid_diameter, 1), slice(7,8,1))
    brute(build_model, grid, args=(series, sord_models, model[0]), finish=None)
  return sord_models

In [8]:
test = grab_air_visit_data_by_id(competition_id_list[0])
mod = sm.tsa.statespace.SARIMAX(
    test['visitors'], trend='n', order=(1,0,0), seasonal_order=(1,0,0,7)
  )
res = mod.fit(disp=0)

In [10]:
#res.predict()

# Model optimization with `parameter_search2`

In [ ]:
mods = parameter_search2(test['visitors'], 5)

In [ ]:
mods

In [ ]:
mod = sm.tsa.statespace.SARIMAX(test['visitors'], trend='n', order=(0, 1, 2), seasonal_order=(0,1,2,7))
best_model = mod.fit(disp=0)

# Build best model from optimization and compare to true

In [ ]:
preds = best_model.predict()
test = test.assign(predictions=preds.values)
test[['predictions','visitors']].plot(figsize=(20,10))